In [3]:
import os
import sys
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from sklearn.impute import SimpleImputer
#parent_directory = os.path.abspath(os.path.join(os.path.dirname(__file__), os.pardir))
#sys.path.append(parent_directory)
from MNAR.missing_process.block_rules import *
import torch
import torch.nn as nn
import numpy as np
import scipy.io
import scipy.sparse
from scipy.io import loadmat
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.experimental import enable_iterative_imputer
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import BayesianRidge
from sklearn.impute import IterativeImputer
from sklearn.impute import SimpleImputer
from MIWAE_imputer_utility import load_dataloader as MIWAE_loader,rmse
from GAIN_imputer_utility import load_dataloader as GAIN_loader

#["Q1_complete","Q1_partial","Q2_complete","Q2_partial","Q3_complete","Q3_partial","Q4_complete","Q4_partial",
#"Q1_Q2_complete","Q1_Q2_partial","Q1_Q3_complete","Q1_Q3_partial","Q1_Q4_complete","Q1_Q4_partial","Q2_Q3_complete","Q2_Q3_partial",
#"Q2_Q4_complete","Q2_Q4_partial","Q3_Q4_complete","Q3_Q4_partial"]

bs = 64    

In [5]:

dataset_file = "banknote"
missing_type = "quantile"
missing_rule = ["Q1_complete"]

In [6]:
def run(dataset_file,missing_name):


    Xtrain, Xtest, Xtrain_mask, Xtest_mask, n, \
        p, x_train_hat_0, x_test_hat_0, x_train_hat,\
            x_test_hat,X_train_miss,X_test_miss = MIWAE_loader(dataset_file,missing_type = "quantile", \
                                        missing_name = missing_name,seed = 1)


    #missforest = IterativeImputer(max_iter=20, estimator=ExtraTreesRegressor(n_estimators=100))
    #iterative_ridge = IterativeImputer(max_iter=20, estimator=BayesianRidge())
    mean_imp = SimpleImputer(missing_values=np.nan, strategy='mean')
    
    # Train
    #missforest.fit(X_train_miss)
    #iterative_ridge.fit(X_train_miss)
    mean_imp.fit(X_train_miss)

    # Test set Evaluation
    # xhat_mf = missforest.transform(X_test_miss)
    # mf_test = rmse(xhat_mf, Xtest, Xtest_mask)

    # xhat_ridge = iterative_ridge.transform(X_test_miss)
    # ridge_test = rmse(xhat_ridge, Xtest, Xtest_mask)

    xhat_mean = mean_imp.transform(X_test_miss)
    mean_test = rmse(xhat_mean, Xtest, Xtest_mask)



    # print("MF Test RMSE:",mf_test)
    # print("Ridge Test RMSE:",ridge_test)
    print("Mean Test RMSE:",mean_test)
    #print("MIWAE Test RMSE:",MIWAE_test)



for missing_name in missing_rule:

    
    results = run(dataset_file, missing_name)


Mean Test RMSE: 0.8197344966447118


In [7]:

def run(dataset_file,missing_rule, use_BN=True):
    
    Imputer_RMSE = []
    baseline_RMSE = []

    for rule_name in missing_rule:
        
        trainX, testX, train_Mask, test_Mask, train_input, test_input, No, Dim, train_H, test_H = GAIN_loader(dataset_file,missing_type, rule_name)

        print(trainX)
        print(testX.shape)
        print(testX)
        print(train_Mask)

        train_nan = trainX.copy()
        test_nan = testX.copy()
        train_nan[train_Mask == 0] = np.nan
        test_nan[test_Mask == 0] = np.nan

        print(train_nan)
        # ------------------------------------------------------------------------------
        imp = SimpleImputer(missing_values=np.nan, strategy='mean')
        imp.fit(train_nan)
        train_imp = imp.transform(train_nan)
        test_imp = imp.transform(test_nan)
        #train_rmse = np.sqrt(np.sum((trainX - train_imp) ** 2 * (1 - train_mask)) / np.sum(1 - train_mask))
        test_rmse = np.sqrt(np.sum((testX - test_imp) ** 2 * (1 - test_Mask)) / np.sum(1 - test_Mask))


        print("Mean Imputer test_rmse:",test_rmse)


    #     baseline_RMSE.append(round(test_rmse,5))


    # result = pd.DataFrame({"Missing_Rule":[rule_name for rule_name in missing_rule],"Imputer RMSE":Imputer_RMSE,"Baseline RMSE":baseline_RMSE})
    # result.to_csv("results/{}_Baseline.csv".format(dataset_file),index=False)


run(dataset_file,missing_rule)

[[-0.56760557  0.59034802  0.05861238 -1.90294239]
 [ 0.26070953 -0.42519136  0.49075943 -0.08670823]
 [ 0.35623912  0.41428941 -0.21852974 -0.64826788]
 ...
 [ 0.3377211   0.28777346  0.06490537  0.2472822 ]
 [-0.12970299  0.33597579 -0.0495842  -0.83832435]
 [-0.53242256  0.64819086  0.05517213 -1.01414698]]
(275, 4)
[[-0.13476384 -0.70729521  0.55436035 -0.11932164]
 [ 0.21788417  0.08398607 -0.04152181  0.39460499]
 [ 0.45372404 -0.28325066  0.38953579  0.16931149]
 ...
 [ 0.45621612 -0.24424438  0.3690734   0.31250962]
 [ 0.59099276  0.17638836  0.12416391  0.84893462]
 [-0.3523796  -0.52263541  0.59718712  0.400229  ]]
[[0 1 1 0]
 [1 0 1 1]
 [1 1 0 0]
 ...
 [1 1 1 1]
 [0 1 0 0]
 [0 1 1 0]]
[[        nan  0.59034802  0.05861238         nan]
 [ 0.26070953         nan  0.49075943 -0.08670823]
 [ 0.35623912  0.41428941         nan         nan]
 ...
 [ 0.3377211   0.28777346  0.06490537  0.2472822 ]
 [        nan  0.33597579         nan         nan]
 [        nan  0.64819086  0.055172